In [435]:
import datetime as dt
import numpy as np
import pandas as pd
import requests
import json
from pprint import pprint
import csv
# from api_key import api_key
from flask import Flask, jsonify, render_template

from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import sqlalchemy

In [436]:
api_key = "93ba5366cb39b249dcbde95cc4d461e7b05b747d"
export_url = (f"https://api.census.gov/data/timeseries/intltrade/exports")
import_url = (f"https://api.census.gov/data/timeseries/intltrade/imports")
china_code = "5700"
usa_code = "1000"
month = []
year = []
#&ParameterName={parameter_name}
#url = (f"https://apps.bea.gov/api/data?&UserID={api_key}&method={get_data}&datasetname={dataset}&LineCode={line_code}&TableName={table_name}&GeoFIPS={geofip}&Year={year}")

#SAINC51
#response = requests.get(url).json()       
#list_response = requests.get(list_url).json()

In [38]:
total_trade = requests.get(f"{export_url}/hs?get=ALL_VAL_MO&key={api_key}&time=2019-01").json() 

In [39]:
total_trade[1][0]


'129545568405'

In [40]:
requests.get(f"https://api.census.gov/data/timeseries/intltrade/exports/hs?get=DF,ALL_VAL_MO,ALL_VAL_YR&YEAR=2013&MONTH=12").json()

[['DF', 'ALL_VAL_MO', 'ALL_VAL_YR', 'YEAR', 'MONTH'],
 ['-', '131915459503', '1578516879950', '2013', '12'],
 ['1', '114956497285', '1371134061205', '2013', '12'],
 ['2', '16958962218', '207382818745', '2013', '12']]

In [160]:
china_exports = requests.get(f"{export_url}/hs?get=ALL_VAL_MO,ALL_VAL_YR&key={api_key}&time=from+2015-01&CTY_CODE={china_code}").json()


In [44]:
# requests.get(f"{export_url}/hs?get=CTY_CODE,CTY_NAME,ALL_VAL_MO,ALL_VAL_YR,E_COMMODITY_LDESC&key={api_key}&time=from+2015-01&CTY_CODE={china_code}").json()


clear


In [45]:
# export_by_commodity = requests.get(f"{export_url}/hs?get=E_COMMODITY,E_COMMODITY_SDESC,ALL_VAL_MO,ALL_VAL_YR&key={api_key}&time=from+2015-01&COMM_LVL=HS4&CTY_CODE={china_code}").json()


In [47]:
len(export_by_commodity)

54073

In [48]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 54072:
    count += 1
    movalue.append(export_by_commodity[count][2])
    yrvalue.append(export_by_commodity[count][3])
    desc.append(export_by_commodity[count][1])
    period.append(export_by_commodity[count][4])
    code.append(export_by_commodity[count][0])
china_data = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value " : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [51]:

china_data.tail()

,YTD Value,Monthly Value,Period,Commodity,Comm Code
54066,320405,37821,2018-08,"MONOPODS, BIPODS, TRIPODS AND SIMILAR ARTICLES",9620
54067,327518,7113,2018-09,"MONOPODS, BIPODS, TRIPODS AND SIMILAR ARTICLES",9620
54068,361042,33524,2018-10,"MONOPODS, BIPODS, TRIPODS AND SIMILAR ARTICLES",9620
54069,399482,38440,2018-11,"MONOPODS, BIPODS, TRIPODS AND SIMILAR ARTICLES",9620
54070,444867,45385,2018-12,"MONOPODS, BIPODS, TRIPODS AND SIMILAR ARTICLES",9620


In [462]:
engine = create_engine('sqlite:///../data/china_trade.sqlite', echo=False)


In [15]:
# china_data.to_sql('export', con=engine, if_exists='replace') #dont run this again

NameError: name 'china_data' is not defined

In [16]:
import_by_commodity = requests.get(f"{import_url}/hs?get=I_COMMODITY,I_COMMODITY_SDESC,GEN_VAL_MO,GEN_VAL_YR&key={api_key}&time=from+2015-01&COMM_LVL=HS4&CTY_CODE={china_code}").json()

In [19]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 55352:
    count += 1
    movalue.append(import_by_commodity[count][2])
    yrvalue.append(import_by_commodity[count][3])
    desc.append(import_by_commodity[count][1])
    period.append(import_by_commodity[count][4])
    code.append(import_by_commodity[count][0])
import_china_data = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value " : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [29]:
import_china_data.to_sql('import', con=engine, if_exists='replace') 

In [24]:
# import_china_data

In [60]:
engine = create_engine('sqlite:///../data/china_trade.sqlite')
#engine = create_engine("sqlite:///db/china_trade.sqlite")




In [467]:
Base = automap_base()
Base.prepare(engine, reflect=True)
session = Session(engine)


In [468]:
Base.classes.keys()

['export', 'hs2export', 'hs2import', 'imports', 'yrhs2export', 'yrhs2import']

In [63]:
Imports = Base.classes.imports

In [474]:
IndImports = Base.classes.hs2import
IndExports = Base.classes.hs2export
YRImports = Base.classes.yrhs2import
YRExports = Base.classes.yrhs2export

In [65]:
Exports = Base.classes.export

In [66]:
exports = session.query(Exports).first()
exports.__dict__


{'Description': 'FISH, DRIED, SALTED ETC, SMOKED ETC; ED FISH MEAL',
 'HSC': 305,
 'MoValue ': 274324,
 'Period': '2015-02',
 'YTDValue': 310025,
 '_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1ea322345c0>,
 'index': 0}

In [14]:
results = session.query(Imports).statement


In [12]:
df = pd.read_sql_query(results, session.bind)

In [13]:
df.columns

Index(['index', 'YTDValue', 'MoValue', 'Period', 'Description', 'HSC'], dtype='object')

In [14]:
list(df.columns)

['index', 'YTDValue', 'MoValue', 'Period', 'Description', 'HSC']

In [13]:
sel = [
        Imports.Description,
        Imports.YTDValue,
        Imports.MoValue,
        Imports.Period,
        Imports.Description
    ]

AttributeError: type object 'imports' has no attribute 'MoValue'

In [3]:
# session.query(*sel).filter(Imports.HSC == "305").all()

In [27]:
stmt = session.query(Imports).statement
df = pd.read_sql_query(stmt, session.bind)


In [120]:
##### GOOD CODE FOR INDIVIDUAL SEARCHES IE TOOLTIP"
year = 2018
hsc = 97
sel = [
        Imports.Description,
        Imports.YTDValue,
        Imports.Period,
        Imports.HSC
        ]

results = session.query(*sel).filter(Imports.HSC == hsc).filter(Imports.Period.like(f"%{year}%")).all()
hsc_data = {}
for result in results:
    hsc_data["Description"] = result[0]
    hsc_data["Annual Value"] = result[1]
    hsc_data["Year"] = result[2]
    hsc_data["HSC"] = result[3]

In [137]:
hsc_data

{'Annual Value': 252327033,
 'Description': 'SPECIAL IMPORT PROVISIONS, NESOI',
 'HSC': 99,
 'Year': '2015-01'}

In [451]:
year_hs2 = requests.get(f"{import_url}/hs?get=I_COMMODITY,I_COMMODITY_SDESC,GEN_VAL_YR&YEAR=2015&YEAR=2016&YEAR=2017&YEAR=2018&YEAR=2019&COMM_LVL=HS2&CTY_CODE={china_code}").json()


In [458]:
len(year_hs2)

4803

In [453]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 4802:
    count += 1
    yrvalue.append(year_hs2[count][2])
    desc.append(year_hs2[count][1])
    period.append(year_hs2[count][3])
    code.append(year_hs2[count][0])
yr_hs2_import = pd.DataFrame({"YTDValue" : yrvalue,
                          "Period" : period,
                          "Description" : desc,
                          "HSC": code})

In [454]:
yr_hs2_import.to_sql('yrhs2import', con=engine, if_exists='replace') 

In [455]:
year_hs2exp = requests.get(f"{export_url}/hs?get=E_COMMODITY,E_COMMODITY_SDESC,ALL_VAL_YR&YEAR=2015&YEAR=2016&YEAR=2017&YEAR=2018&YEAR=2019&COMM_LVL=HS2&CTY_CODE={china_code}").json()


In [456]:
len(year_hs2exp)

4746

In [460]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 4745:
    count += 1
    yrvalue.append(year_hs2exp[count][2])
    desc.append(year_hs2exp[count][1])
    period.append(year_hs2exp[count][3])
    code.append(year_hs2exp[count][0])
yr_hs2_export = pd.DataFrame({"YTDValue" : yrvalue,
                          "Period" : period,
                          "Description" : desc,
                          "HSC": code})

In [461]:
yr_hs2_export.to_sql('yrhs2export', con=engine, if_exists='replace') 

In [19]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 4802:
    count += 1
    yrvalue.append(import_hs2[count][2])
    desc.append(import_hs2[count][1])
    period.append(import_hs2[count][3])
    code.append(import_hs2[count][0])
hs2_import = pd.DataFrame({"YTDValue" : yrvalue,
                          "Period" : period,
                          "Description" : desc,
                          "HSC": code})


In [21]:
# hs2_import.to_sql('hs2import', con=engine, if_exists='replace') 

In [33]:
# export_hs2 = requests.get(f"{export_url}/hs?get=E_COMMODITY,E_COMMODITY_SDESC,ALL_VAL_MO,ALL_VAL_YR&time=from+2015-01&COMM_LVL=HS2&CTY_CODE={china_code}").json()



In [143]:
# count = 1
# movalue = []
# yrvalue = []
# period = []
# desc = []
# code = []
# while count < 4802:
#     count += 1
#     movalue.append(import_by_commodity[count][2])
#     yrvalue.append(import_by_commodity[count][3])
#     desc.append(import_by_commodity[count][1])
#     period.append(import_by_commodity[count][4])
#     code.append(import_by_commodity[count][0])
# hs2_export = pd.DataFrame({"YTDValue" : yrvalue,
#                            "MoValue " : movalue,
#                           "Period" : period,
#                           "Description" : desc,
#                           "HSC": code})


In [34]:
# hs2_export.to_sql('hs2export', con=engine, if_exists='replace') 


In [ ]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 4802:
    count += 1
    yrvalue.append(import_hs2[count][2])
    desc.append(import_hs2[count][1])
    period.append(import_hs2[count][3])
    code.append(import_hs2[count][0])
hs2_import = pd.DataFrame({"YTDValue" : yrvalue,
                          "Period" : period,
                          "Description" : desc,
                          "HSC": code})

In [505]:
stmt = session.query(IndImports).statement
df = pd.read_sql_query(stmt, session.bind)

In [506]:
df

,index,YTDValue,Period,Description,HSC
0,0,1395030,2015-02,LIVE ANIMALS,1
1,1,4249990,2015-03,LIVE ANIMALS,1
2,2,8561570,2015-04,LIVE ANIMALS,1
3,3,9446570,2015-05,LIVE ANIMALS,1
4,4,13469407,2015-06,LIVE ANIMALS,1
5,5,14980503,2015-07,LIVE ANIMALS,1
6,6,18290205,2015-08,LIVE ANIMALS,1
7,7,21325543,2015-09,LIVE ANIMALS,1
8,8,25069663,2015-10,LIVE ANIMALS,1
9,9,26831863,2015-11,LIVE ANIMALS,1


In [503]:
stmt = session.query(IndImports).statement
df = pd.read_sql_query(stmt, session.bind)

year = 2015
hsc = 1
sel = [
        IndImports.Description,
        IndImports.YTDValue,
        IndImports.Period,
        IndImports.HSC
        ]

results = session.query(*sel).filter(IndImports.HSC == 1).filter(IndImports.Period.like(f"%{year}%")).all()
hsc_ind_imports = {}
for result in results:
    hsc_ind_imports["Description"] = result[0]
    hsc_ind_imports["Annual Value"] = result[1]
    hsc_ind_imports["Year"] = result[2]
    hsc_ind_imports["HSC"] = result[3]

In [178]:
g = session.query(IndImports.HSC).filter(IndImports.HSC == "01").all()

In [504]:
hsc_ind_imports

{'Annual Value': 27704684,
 'Description': 'LIVE ANIMALS',
 'HSC': 1,
 'Year': '2015-12'}

In [297]:
yr = session.query(YRImports).first()
yr.__dict__

{'Description': 'LEAD AND ARTICLES THEREOF',
 'HSC': 78,
 'Period': '2017',
 'YTDValue': 2194573,
 '_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1ea339e82e8>,
 'index': 0}

In [407]:
stmt = session.query(YRImports).statement
df = pd.read_sql_query(stmt, session.bind)

In [398]:
# df.loc[df['Period'] == 2016]

In [249]:
test_df = df.groupby(["Period"])["YTDValue", "HSC", "Period"]

In [332]:
df["YTDValue", "HSC", "Period"]= [i[:4] for i in df['Period']]

In [409]:
df = pd.read_sql_query(stmt, session.bind)

In [408]:
df["Period"]

TypeError: 'Series' object is not callable

In [399]:
yr_hs2_import["YTDValue"] =pd.to_numeric(yr_hs2_import["YTDValue"])


In [400]:
year = "2015"
first_2015 = yr_hs2_import[yr_hs2_import["Period"].str.contains(f"{year}")]


In [502]:
check = first_2015.groupby(["HSC","Description","Period"])["YTDValue"].sum()


In [402]:
test= pd.DataFrame({"total" : data_2015})
data_2015= test.nlargest(10,"total")
data_2015 = data_2015.reset_index()

data_2015= data_2015.nlargest(10,"total")

In [403]:
data_2015

,HSC,Description,Period,total
0,85,ELECTRIC MACHINERY ETC; SOUND,2015,818112819400
1,84,"NUCLEAR REACTORS, BOILERS, MACHINERY ETC.; PARTS",2015,666100643226
2,94,FURNITURE; BEDDING ETC; LAMPS NESOI ETC,2015,181010404192
3,95,"TOYS, GAMES & SPORT EQUIPMENT; PARTS & ACCESSO...",2015,137998619706
4,64,"FOOTWEAR, GAITERS ETC. AND PARTS THEREOF",2015,113801708948
5,61,"APPAREL ARTICLES AND ACCESSORIES, KNIT OR CROCHET",2015,97092844954
6,62,"APPAREL ARTICLES AND ACCESSORIES, NOT KNIT ETC.",2015,94814507202
7,39,PLASTICS AND ARTICLES THEREOF,2015,94005729937
8,87,"VEHICLES, EXCEPT RAILWAY OR TRAMWAY, AND PARTS...",2015,85574454525
9,90,"OPTIC, PHOTO ETC, MEDIC OR S",2015,71151180495


In [495]:
year= "2016"
stmt = session.query(YRExports).statement
df = pd.read_sql_query(stmt, session.bind)
df["YTDValue"] =pd.to_numeric(df["YTDValue"])
first_2015 = df[df["Period"].str.contains(f"{year}")]
data_2015 = first_2015.groupby(["HSC","Description","Period"])["YTDValue"].sum()
test= pd.DataFrame({"total" : data_2015})
data_2015= test.nlargest(10,"total")
data_2015 = data_2015.reset_index()

data_2015= data_2015.nlargest(10,"total")

In [476]:
data_2015= data_2015.to_json()


In [496]:
data_2015

,HSC,Description,Period,total
0,88,"AIRCRAFT, SPACECRAFT, AND PARTS THEREOF",2016,86651134392
1,85,ELECTRIC MACHINERY ETC; SOUND,2016,78930329601
2,84,"NUCLEAR REACTORS, BOILERS, MACHINERY ETC.; PARTS",2016,74341211586
3,87,"VEHICLES, EXCEPT RAILWAY OR TRAMWAY, AND PARTS...",2016,68051586710
4,12,"OIL SEEDS ETC.; MISC GRAIN, SEED, FRUIT, PLANT...",2016,67929422483
5,90,"OPTIC, PHOTO ETC, MEDIC OR S",2016,52258343745
6,39,PLASTICS AND ARTICLES THEREOF,2016,30831182836
7,47,WOOD PULP ETC; RECOVD (WASTE & SCRAP) PPR & PPRBD,2016,21010065898
8,44,WOOD AND ARTICLES OF WOOD; WOOD CHARCOAL,2016,16010520349
9,29,ORGANIC CHEMICALS,2016,15204269108


In [497]:
year= "2016"
stmt = session.query(YRImports).statement
df = pd.read_sql_query(stmt, session.bind)
df["YTDValue"] =pd.to_numeric(df["YTDValue"])
first_2015 = df[df["Period"].str.contains(f"{year}")]
data_2015 = first_2015.groupby(["HSC","Description","Period"])["YTDValue"].sum()
test= pd.DataFrame({"total" : data_2015})
data_2015= test.nlargest(10,"total")
data_2015 = data_2015.reset_index()

imp_2015= data_2015.nlargest(10,"total")

In [501]:
imp_2015["total"].div(100000)

0    7.727439e+06
1    6.185776e+06
2    1.863899e+06
3    1.356311e+06
4    9.768266e+05
5    9.259231e+05
6    8.961216e+05
7    8.787688e+05
8    8.654447e+05
9    7.121349e+05
Name: total, dtype: float64

In [424]:
pie_imports = {}
for result in data_2015:
    pie_imports["Description"] = result[1]
    pie_imports["Annual Value"] = result[2]
    pie_imports["HSC"] = result[0]

In [429]:
data_2015.to_dict("records")

[{'Description': 'ELECTRIC MACHINERY ETC; SOUND',
  'HSC': '85',
  'Period': '2015',
  'total': 818112819400},
 {'Description': 'NUCLEAR REACTORS, BOILERS, MACHINERY ETC.; PARTS',
  'HSC': '84',
  'Period': '2015',
  'total': 666100643226},
 {'Description': 'FURNITURE; BEDDING ETC; LAMPS NESOI ETC',
  'HSC': '94',
  'Period': '2015',
  'total': 181010404192},
 {'Description': 'TOYS, GAMES & SPORT EQUIPMENT; PARTS & ACCESSORIES',
  'HSC': '95',
  'Period': '2015',
  'total': 137998619706},
 {'Description': 'FOOTWEAR, GAITERS ETC. AND PARTS THEREOF',
  'HSC': '64',
  'Period': '2015',
  'total': 113801708948},
 {'Description': 'APPAREL ARTICLES AND ACCESSORIES, KNIT OR CROCHET',
  'HSC': '61',
  'Period': '2015',
  'total': 97092844954},
 {'Description': 'APPAREL ARTICLES AND ACCESSORIES, NOT KNIT ETC.',
  'HSC': '62',
  'Period': '2015',
  'total': 94814507202},
 {'Description': 'PLASTICS AND ARTICLES THEREOF',
  'HSC': '39',
  'Period': '2015',
  'total': 94005729937},
 {'Description':